In [ ]:
import pandas as pd
import glob

# List of file names corresponding to the stocks
file_names = ["AAPL_historical_data.csv", "AMZN_historical_data.csv", "GOOG_historical_data.csv", 
              "META_historical_data.csv", "MSFT_historical_data.csv", "NVDA_historical_data.csv", 
              "TSLA_historical_data.csv"]

# Initialize an empty dictionary to store DataFrames
stock_data = {}

# Load each file into a DataFrame and store in the dictionary
for file_name in file_names:
    # Extract stock symbol from the file name (e.g., "AAPL" from "AAPL_historical_data.csv")
    stock_symbol = file_name.split("_")[0]
    
    # Load the CSV file
    df = pd.read_csv(file_name)
    
    # Convert the Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Set the Date column as the index
    df.set_index('Date', inplace=True)
    
    # Store the DataFrame in the dictionary
    stock_data[stock_symbol] = df

# Check the dictionary keys to verify the stocks loaded
print(stock_data.keys())


In [ ]:
# Initialize an empty list to store modified DataFrames
df_list = []

for stock_symbol, df in stock_data.items():
    # Add a column to indicate the stock symbol
    df['Stock'] = stock_symbol
    
    # Append the DataFrame to the list
    df_list.append(df)

# Concatenate all DataFrames into a single DataFrame
merged_df = pd.concat(df_list)

# Display the first few rows of the merged DataFrame
merged_df.head()


In [ ]:
# Example: Filter data for Apple (AAPL)
aapl_data = merged_df[merged_df['Stock'] == 'AAPL']

# Example: Display the merged DataFrame
print(merged_df.head())


In [ ]:
import talib as ta
import seaborn as sns
import matplotlib.pyplot as plt

# Initialize an empty dictionary to store the indicators
indicators = {}

# Compute indicators for each stock in the merged DataFrame
for stock_symbol in merged_df['Stock'].unique():
    stock_df = merged_df[merged_df['Stock'] == stock_symbol]
    
    # Moving Averages
    stock_df['SMA_20'] = ta.SMA(stock_df['Close'], timeperiod=20)
    stock_df['SMA_50'] = ta.SMA(stock_df['Close'], timeperiod=50)
    
    # RSI (Relative Strength Index)
    stock_df['RSI'] = ta.RSI(stock_df['Close'], timeperiod=14)
    
    # MACD (Moving Average Convergence Divergence)
    stock_df['MACD'], stock_df['MACD_Signal'], stock_df['MACD_Hist'] = ta.MACD(
        stock_df['Close'], 
        fastperiod=12, 
        slowperiod=26, 
        signalperiod=9
    )
    
    # Store the calculated indicators back to the dictionary
    indicators[stock_symbol] = stock_df

# Example: Display the indicators for AAPL
indicators['AAPL'].head()


In [ ]:
# Plot the Moving Averages for AAPL
plt.figure(figsize=(14, 7))
plt.plot(indicators['AAPL'].index, indicators['AAPL']['Close'], label='Close Price')
plt.plot(indicators['AAPL'].index, indicators['AAPL']['SMA_20'], label='20-Day SMA')
plt.plot(indicators['AAPL'].index, indicators['AAPL']['SMA_50'], label='50-Day SMA')
plt.title('AAPL Close Price and Moving Averages')
plt.legend()
plt.show()


In [ ]:
# Plot the RSI for AAPL
plt.figure(figsize=(14, 7))
plt.plot(indicators['AAPL'].index, indicators['AAPL']['RSI'], label='RSI')
plt.axhline(70, color='red', linestyle='--', label='Overbought (70)')
plt.axhline(30, color='green', linestyle='--', label='Oversold (30)')
plt.title('AAPL RSI')
plt.legend()
plt.show()


In [ ]:
# Plot the MACD for AAPL
plt.figure(figsize=(14, 7))
plt.plot(indicators['AAPL'].index, indicators['AAPL']['MACD'], label='MACD')
plt.plot(indicators['AAPL'].index, indicators['AAPL']['MACD_Signal'], label='MACD Signal')
plt.bar(indicators['AAPL'].index, indicators['AAPL']['MACD_Hist'], label='MACD Histogram')
plt.title('AAPL MACD')
plt.legend()
plt.show()
